In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
50,Here's a real weirdo for you. It starts out wi...,positive
290,This romantic comedy isn't too bad. There are ...,negative
78,"""Showtime"" falls somewhere north of what some ...",positive
998,Nothing but the director's juvenile fantasy co...,negative
859,Jonathan Demme's directorial debut for Roger C...,positive


In [4]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
import nltk
nltk.download("stopwards")
nltk.download("wordnet")
df = normalize_text(df)
df.head()

[nltk_data] Error loading stopwards: Package 'stopwards' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review,sentiment
50,here s real weirdo you start another take off ...,positive
290,romantic comedy bad funny thing happening ther...,negative
78,showtime fall somewhere north critic saying li...,positive
998,nothing director s juvenile fantasy come life ...,negative
859,jonathan demme s directorial debut roger corma...,positive


In [8]:
df['sentiment'].value_counts()

sentiment
negative    253
positive    247
Name: count, dtype: int64

In [9]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [10]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
50,here s real weirdo you start another take off ...,1
290,romantic comedy bad funny thing happening ther...,0
78,showtime fall somewhere north critic saying li...,1
998,nothing director s juvenile fantasy come life ...,0
859,jonathan demme s directorial debut roger corma...,1


In [11]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [16]:
y

50     1
290    0
78     1
998    0
859    1
      ..
874    1
84     0
72     1
63     0
776    0
Name: sentiment, Length: 500, dtype: int64

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/2017naroju/mlops-capstone.mlflow')
dagshub.init(repo_owner='2017naroju', repo_name='mlops-capstone', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/home/azureuser/.local/lib/python3.10/site-packages/rich/live.py:229: UserWarning: install "ipywidgets" for Jupyter
support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=526647d9-bb47-4003-b2b8-780ca529309a&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=75e8433d70dbe27a72f1ac562678fd03a974ea0aa8f78781c145c19ec429ce9a




Accessing as 2017naroju

Initialized MLflow to track repo "2017naroju/mlops-capstone"

Repository 2017naroju/mlops-capstone initialized!

2025/03/04 10:47:37 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/71883f39ca9b477daed0d4d5bf9d1edd', creation_time=1741085257876, experiment_id='0', last_update_time=1741085257876, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [21]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 150)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-04 10:52:51,827 - INFO - Starting MLflow run...
2025-03-04 10:52:51,928 - INFO - Logging preprocessing parameters...
2025-03-04 10:52:52,179 - INFO - Initializing Logistic Regression model...
2025-03-04 10:52:52,180 - INFO - Fitting the model...
2025-03-04 10:52:52,196 - INFO - Model training complete.
2025-03-04 10:52:52,197 - INFO - Logging model parameters...
2025-03-04 10:52:52,293 - INFO - Making predictions...
2025-03-04 10:52:52,294 - INFO - Calculating evaluation metrics...
2025-03-04 10:52:52,302 - INFO - Logging evaluation metrics...
2025-03-04 10:52:52,678 - INFO - Saving and logging the model...
2025/03/04 10:52:59 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-03-04 10:52:59,684 - INFO - Model training and logging completed in 7.76 seconds.
2025-03-04 10:52:59,685 - INFO - Accuracy: 0.648
2025-03-04 10:52:59,687 - INFO - Precision: 0.6101694915254238
2025-03-